# Predictions of fraudlent users in a Bitcoin user network #

On platforms where users interact with each other to perform some type of financial transaction, it is very desirable to stop users who act in a fraudulent manner from being able to access or continue to use the platform. However, investigating users can be very expensive (consider e.g sellers on Ebay or Amazon) or impossible (consider users on a Bitcoin network) depending on the context. It is therefore worth asking the following - is it possible to perform accurate inference for the trustworthiness of users simply based on their behavior on the platform? We investigate this using publically available data for a Bitcoin user platform, where we observe ratings between users (and the times at which these ratings are made), and only know for a small subset of users whether they are trustworthy or not. 

In [1]:
### DEMO - switch to True for checking the code works. 
demo = False

## Summary of data ## 

For the dataset we are considering, we have 5,881 nodes (corresponding to separate users) and 35,592 weighted edges (corresponding to ratings from one user to another), along with time-stamp data; for example, a subset of the data looks like the following:

Source node | Target node | Rating | Time stamp (seconds after Epoch)
:----------:|:-----------:|:------:|:---------------------------------:
6 | 2 |	4 | 1289241912
6 | 5 | 2 | 1289241942
1 | 15 | 1 | 1289243140
4 | 3 | 7 | 1289245277
13 | 16 | 8 | 1289254254
13 | 10 | 8 | 1289254301
7 | 5 | 1 | 1289362700
2 | 21 | 5 | 1289370557

When analysing the data, we transform them to lie in $[0, 1]$ and treat the ratings as continuous. We also have some notion of "ground truth" data for the users in the network - there are 136 users which are considered trustworthy, and 180 not trustworthy users (so we have ground truth data for 316 users in total). Note that this is only for a subset of the users, so we will use a semi-supervised approach to modelling our data. We will split this in half to give a test and a training set for semi-supervised learning.

## A quick caveat on our notion of ground truth ## 

Given that our data is taken from an inherently anonymous platform, you may (rightfully) ask whether the notion of 'ground-truth trustworthiness' comes from. For this dataset, the platform founder and those who they rated highly are considered to be trustworthy, and the users rated very lowly by the latter group are considered to be un-trustworthy. As a result, this is a clear limitation as to whether these users are actually acting fraudulently or not - instead we are really making inference for whether the platform holder would consider someone to be trustworthy. From e.g the perspective of the platform founder, this probably would suffice. 

# Exploratory data analysis

Before beginning to model the data in order to try and make inference for the trustworthiness of different users, we begin by exploring some of the features of our dataset to see what could potentially be informative for the purposes of achieving our overall goal.

In [2]:
import warnings
import torch
import logging
import numpy as np

# Used to supress some NumPy warnings when loading data
warnings.simplefilter('ignore', category=RuntimeWarning)

# Printing logging information
logger = logging.getLogger()
logger.setLevel(logging.INFO)

# Used to automatically reload imported scripts for debugging
%load_ext autoreload
%autoreload 1

# Import various external scripts with auto-reloading
%aimport eda
%aimport plotfns
%aimport bayes
%aimport bitcoin
%aimport embed


In [3]:
if demo:
    eda.plot_summary_stat(fname=None, display=True)
else:
    eda.plot_summary_stat(fname='img/summary_stat.png', display=False)


![title](img/summary_stat.png)

Examining the above plots, we see that in terms of node-level information we can use to try and predict whether a user is trustworthy, we observe that there are a few discriminating features we could try and use:

* The average rating received
* The standard deviation of ratings given and received
* The standard deviation of rating times

Given this, we'll therefore begin by seeing what happens when using a simple logistic regression on the labelled part of the dataset, using these as features.

# Model 1: A straightforward Bayesian logistic regression

We'll begin by fitting the following Bayesian logistic regression model based off the examination above: we'll suppose that user $i$ has a trustworthy indicator $a_i$ with $a_i = 1$ corresponding to a user being trustworthy and $a_i = 0$ not trustworthy. We'll then suppose that the $a_i$ are independent Bernoulli random variables, whose probability of success $p_i$ conditional on 

* The average rating received $\mathrm{R}_i$, centered to lie within $[-0.5, 0.5]$;
* The standard deviation of ratings given $\mathrm{SDG}_i$ and received $\mathrm{SDR}_i$, rescaled to lie within $[0, 1]$; and
* The standard deviation of rating times $\mathrm{SDT}_i$ given, rescaled to lie within $[0, 1]$ (achieved via the mapping $x \mapsto \log(x+1)$ and then dividing by the observed maximum).

is given by 

$$ \log\Big( \frac{p_i}{1 - p_i} \Big) = \mu + \beta_R \cdot \mathrm{R}_i + \beta_{SDG} \cdot \mathrm{SDG}_i + \beta_{SDR} \cdot \mathrm{SDR}_i + \beta_{SDT} \cdot \mathrm{SDT}_i. $$

For users with zero ratings, all of the relevant standard deviations can be set to zero, and similarly we can force $R_i$ to be equal to zero for users with no ratings. 

In fitting the above model, we place independent $\mathrm{Normal}(0, 1)$ priors on each of the non-intercept regression parameters (note that as we have normalized our covariates to be on the same scale, we also keep our prior variables on the same scale too), and a $\mathrm{Normal}(0, 10)$ prior on the intercept parameter. We fit the above model using MCMC (note we can only fit this on the data for which we know the ground truth labels).

In [4]:
if demo:
    hmc_arg_dict = {'model_num': 1, 'num_samples': 50, 'warmup_steps': 10}
else:
    hmc_arg_dict = {'model_num': 1, 'num_samples': 1000, 
                    'warmup_steps': 1000}

# Run MCMC to get draws from the posterior distribution
hmc_samples = bayes.bayes_logistic_reg(**hmc_arg_dict)


sample: 100%|█████████████████████████████████| 2000/2000 [04:38<00:00,  7.19it/s, step size=2.90e-01, acc. prob=0.946]


Site: mu
       mean       std        5%      25%       50%       75%     95%
0 -0.681589  0.492239 -1.493878 -0.99971 -0.669956 -0.361989  0.1096 

Site: beta_r
       mean       std        5%       25%       50%       75%       95%
0  4.566194  0.721594  3.444496  4.069345  4.550468  5.024158  5.795468 

Site: beta_sdg
       mean      std        5%       25%       50%       75%       95%
0  0.364244  0.78393 -0.919632 -0.144635  0.376991  0.867515  1.639252 

Site: beta_sdr
       mean       std        5%       25%       50%       75%      95%
0 -1.988887  0.763089 -3.197692 -2.525695 -1.974224 -1.451411 -0.75849 

Site: beta_sdt
       mean       std        5%       25%       50%       75%       95%
0  2.703743  0.588526  1.798399  2.287473  2.683432  3.105769  3.676762 



## Criticism for Model 1 ##

We'll begin by looking at the ROC curves produced by looking at the classifier created by the above model.

In [5]:
# Import data, transform covariates to make test set predictions
# and do some kind of posterior predictive checks
data = bitcoin.BitcoinOTC()

r = data.in_weight_avg - 0.5
sdg = 2*data.out_weight_std
sdr = 2*data.in_weight_std
sdt = torch.log(data.rate_time_out_std + 1)
sdt = sdt/sdt.max()
gt = data.gt

# Set all NaN values to zero
sdg[sdg != sdg] = 0
sdr[sdr != sdr] = 0
sdt[sdt != sdt] = 0

# Create 5*(num of samples) matrix of regression coefficients
reg_coefs = []
for value in hmc_samples.values():
    reg_coefs.append(value)
    
reg_coefs = np.array(reg_coefs)

# Create (num of train/test points)*4 matrix of covariates 
tt_str_list = ['train', 'test']
logits = dict.fromkeys(tt_str_list)

for i, split in enumerate([data.nodes_train, data.nodes_test]):
    # Create (number of data points)*5 array of covariates
    covs = np.array([np.ones(len(split)),
                     r[split].numpy(),
                     sdg[split].numpy(),
                     sdr[split].numpy(),
                     sdt[split].numpy()])
    
    # Multiply to give (num of data points)*(num of samples) array
    logits[tt_str_list[i]] = covs.transpose() @ reg_coefs
    
# Create dict of ground truth labels for test and training set
gt_labels = {'train': gt[data.nodes_train].numpy(),
             'test': gt[data.nodes_test].numpy()}
           
# Create/plot ROC curve for training and test set 
if demo:
    plotfns.plot_roc_curve(gt=gt_labels, logits=logits, display=True)
else:
    plotfns.plot_roc_curve(gt=gt_labels, logits=logits, 
                           fname='img/bayes_1_roc.png', display=False);


![title](img/bayes_1_roc.png)

Here, we use each draw from the posterior distribution to form logit scores for the probability of trustworthiness, calculate the ROC and the corresponding AOC, and then average over our draws from the posterior distribution. Looking at this curve, you may think that we effectively have a perfect classifier, and so our task is done. However, what we are really interested in is obtaining correctly calibrated probabilities of trustworthiness; that is, if we had e.g $100$ users who had estimated trustworthiness probability equal to (say) $0.8$, then approximately $80$ should be trustworthy. If we plot a calibration curve based off the test set data, we see that this isn't the case:

In [6]:
if demo:
     plotfns.plot_calibration_curve(gt=gt_labels, logits=logits,
                                    display=True)
else:
    plotfns.plot_calibration_curve(gt=gt_labels, logits=logits,
                                   fname='img/bayes_1_calib_curve.png', display=False)


![title](img/bayes_1_calib_curve.png)

For our probabilities to be correctly calibrated, they should lie close to the diagonal line - we can see that this is clearly not the case here.

To see some other aspects for which the above approach leaves something to be desired, we plot the distribution of estimated trustworthiness probabilities across the population of users in the network:  

In [7]:
covs = np.array([np.ones(len(r.numpy())), r.numpy(),
                 sdg.numpy(), sdr.numpy(), sdt.numpy()])
pop_logits = covs.transpose() @ reg_coefs

if demo:
    plotfns.plot_logit_distribution(gt=gt, pop_logits=pop_logits,
                                    train_set=data.nodes_train,
                                    test_set=data.nodes_test,
                                    display=True)
else:
    plotfns.plot_logit_distribution(gt=gt, pop_logits=pop_logits,
                                    train_set=data.nodes_train,
                                    test_set=data.nodes_test,
                                    fname='img/bayes_1_logit_pop.png',
                                    display=False)


![title](img/bayes_1_logit_pop.png)

Here the bold blue line corresponds to the empirical c.d.f produced using the posterior mean of the estimated logits. Tick marks are also displayed corresponding to the estimated probabilities for the users which lie in the training set (black lines) or test set (pink asterisks), plotted at $y = 0$ if they are not trustworthy, and $y = 1$ if they are trustworthy. In addition to the threshold probability being low (a more natural threshold would usually be $0.5$), we see that there is a vertical jump in the c.d.f at around $p= 0.4$. This corresponds to the users for which we have little information, and so we would expect this jump to occur at the proportion of trustworthy users in the total population. However, according to the threshold we have, we would expect approximately $10\%$ of users to be not trustworthy, meaning we have a disparity between the two. 

The major shortcoming of this approach can be best described as follows: 

1. **Labelled sub-population is a 'biased' sample from the overall population** - For this logistic regression to make sense, it requires that the users for which we have labelled data are reflective of the population of users as a whole in the network. Given both the nature of which the ground truth data were collected (we effectively only identify the ground truth in a neighbourhood of the platform founder), and the plots we showed before, this is clearly not the case. 

2. **Requires labels for both trustworthy and not-trustworthy users** - This is self-explanatory - note that in practice, if users are anonymous, then identifying whether they are actually trustworthy or not may be effectively impossible. We'd therefore like a method which is either unsupervised, or could at least allow for only observing trustworthy users (for instance, users known to the platform holder externally could be verified to be trustworthy). Ideally, we'd like a method which is able to learn some underlying latent structure from just the observed edge information, which should be naturally correlated to whether users are trustworthy. We can either use this directly, or hope that by adding a small amount of labels (even just from one class) we'll be able to obtain naturally calibrated probabilities of trustworthiness.

# Model 2: Trying to jointly learn user behavior and trustworthiness

Let's now think a bit more about how we would expect trustworthy and non-trustworthy/fraudlent users to act in this network. For trustworthy users, we expect that

* they will tend to vote other trustworthy users as being trustworthy; and
* they will tend to vote trustworthy users as being untrustworthy.

For un-trustworthy users, it is somewhat harder to try and reason as to how they would behave, as they may act in an adversarial manner, and they may not necessarily always act in the same way. We should try and choose a model which allows us some degree of flexibility, while also allowing us to incorporate the ground truth information we have available in a natural fashion.

We therefore posit a model which acts as a mix between a logistic regression for the trustworthiness indicator, and a matrix factorization model for the observed ratings between users. To do so, we form a likelihood

$$ \begin{aligned} 
p(e_1, \ldots, e_N \,|\, \omega_1, \ldots, \omega_N, \beta, \mu) & = \prod_{i=1}^N \mathrm{Bernoulli}\Big( e_i \,\Big|\, p_i = \sigma\big( \mu + \langle \beta,  \omega_i \rangle \big) \Big) \\
p\big( \{a_{ij} \,:\, i \neq j \} \,|\, \omega_1, \ldots, \omega_N \big) & = \prod_{i \neq j} \mathrm{Normal}\big( \mathrm{logit}(a_{ij}) \,\big|\, \langle \omega_i, \omega_j \rangle, \tau_0^2 \big) 
\end{aligned} $$

where we have priors on the $\omega_i$, $\beta$ and $\mu$ as follows:

$$ \begin{aligned}
p(\omega_1, \ldots, \omega_N) & = \prod_{i=1}^N \mathrm{Normal}\Big(\omega_i \,\Big|\, 0, \sigma^2_0 I_d \Big) \\
p(\beta) &= \mathrm{Normal}\big( \beta \,\big|\, 0, 4 I_d \big) \\
p(\mu) &= \mathrm{Normal}( \mu \,|\, 0, 4 )
\end{aligned}
$$

This can be thought of as a tradeoff between a matrix factorization model for the edge ratings, along with a logistic regression for the underlying class labels. Here the variances $\sigma_0^2$, $\tau_0^2$ and dimension $d$ are hyperparameters of the model which we will select before trying to perform (approximate) inference.

To find the posterior $p(\omega_1, \ldots, \omega_N, \beta \,|\, e_i, a_{ij} )$, we will approximate it using the following variational family

$$ \begin{aligned} p(\omega_1, \ldots, \omega_N, \beta, \mu) & = p(\omega_1, \ldots, \omega_N) p(\beta) p(\mu) \qquad \text{ where }\\
p(\omega_1, \ldots, \omega_N ) & = \prod_{i=1}^N \mathrm{Normal}\big( \omega_i \,\big|\, \mu_i, \tau^2 I_d \big) \\
p(\beta) &= \mathrm{Normal}\big( \beta \,\big|\, \beta_0, \sigma_{\beta}^2 I_d \big) \\
p(\mu) &= \mathrm{Normal}\big( \beta \,\big|\, \mu_0, \sigma_{\mu}^2 \big)
\end{aligned} $$

and optimize over $\sigma_{\beta}^2, \sigma_{\mu}^2, \tau^2$ and the $\mu_i \in \mathbb{R}^d$ for $i = 1, \ldots, N$.

A few notes:

* Our original model is invariant to all the $\omega_i$ and $\beta$ being transformed by the same orthogonal matrix - by using the variational approximation, we will be effectively choosing one representative from this equivalence class. Importantly, this does not interfere with our inference for $p_i$.
* Here we have specified a model as if the network were entirely observed, which is not the case in practice. We therefore only include edges which have been actually observed; we note that this only leads to correct inference for the underlying latent states if the way in which edges are formed in the graph is uninformative for these once given the observed values of these edges.

# An aside: Stochastic Variational Inference for Network Data #

Recall that in variational inference, the goal is to approximate a desired posterior distribution $p(\mathbf{z} | \mathbf{x})$ of some latent variables $\mathbf{z}$ given our data $\mathbf{x}$ using a variational family $q(\mathbf{z} | \lambda)$, with $\lambda$ parameterizing our variational family. To try and do so in an automated fashion, we update $\lambda$ via performing gradient ascent on the ELBO, whose gradient is 

$$ \nabla_{\lambda} \mathrm{ELBO} = \mathbb{E}_{\mathbf{z} \sim q(\cdot|\lambda) }\Big[ \nabla_{\lambda} \log q(\mathbf{z} | \lambda) \big\{ \log p(\mathbf{x}, \mathbf{z}) - \log q(\mathbf{z} | \lambda) \big\} \Big]   $$

In practice, we then usually make use of the two following ideas in order to perform computationally efficient inference - we approximate the expectation by drawing samples from $\mathbf{z} \sim q(\cdot | \lambda)$, and then we also try and approximate the log-likelihood and its derivative. For the latter, the approach of subsampling data points is very intuitive for when we are fitting data which either naturally arises as being conditionally independent.

However, for certain types of structured data, some care and thought is required in trying to exploit subsampling ideas to approximate the log-likelihood. Consider the following model: we have a directed graph $\mathcal{G} = (\mathcal{V}, \mathcal{E})$ without self loops, where $|\mathcal{V}| = N$, and $\mathcal{E}$ is the list of edges, and have

$$ \begin{aligned}
p(\omega_1, \ldots, \omega_N) &= \prod_{i=1}^N N(\omega_i \,|\, 0, \eta^2 I_d) \\
p(\{ a_{ij} \,|\, i \neq j \} \,|\, \omega_1, \ldots, \omega_n) &= \prod_{i \neq j} \mathrm{Bernoulli}\big(a_{ij} \,\big|\, \sigma(\langle \omega_i, \omega_j \rangle) \big)
\end{aligned}$$

where $\eta^2$ is fixed, and $\sigma(x) = e^x/(1+e^x)$ is the logistic function. This has log-likelihood

$$
p\big( \{ \omega_i \}_i, \{ a_{ij} \}_{i \neq j} \big) = \sum_{i \neq j} \Big\{ a_{ij} \log \sigma (\langle \omega_i, \omega_j \rangle) + (1 - a_{ij}) \log\big( 1 - \sigma(\langle u, v \rangle ) \big) \Big\} - \sum_{i=1}^n \frac{1}{2\sigma^2} \| \omega_i \|_2^2. 
$$

If we now let $S(\mathcal{G}) = (\mathcal{V}', \mathcal{E}')$ denoted a "subsampled" version of the network, where $\mathcal{V}' \subseteq \mathcal{V}$ and $\mathcal{E}'$ is a list of pairs of vertices $\mathcal{E}'$ (note that this may necessarily not be a subset of $\mathcal{E}$ of our original edge set), we could approximate the log-likelihood by 

$$
p\big( \{ \omega_i \}_i, \{ a_{ij} \}_{i \neq j} \big) \approx \frac{ N(N-1)  }{ |\mathcal{E}'| } \sum_{(i, j) \in \mathcal{E}'} \Big\{ a_{ij} \log \sigma (\langle \omega_i, \omega_j \rangle) + (1 - a_{ij}) \log\big( 1 - \sigma(\langle u, v \rangle ) \big) \Big\} - \frac{ N }{ |\mathcal{V}'|   } \sum_{i \in \mathcal{V}'} \frac{1}{2\sigma^2} \| \omega_i \|_2^2. 
$$

The question is then in selecting $S(\mathcal{G})$ - for example, this can be chosen by performing random walks on the graph, or by selecting edges/nodes randomly. Vietch et al. [1] highlights that the **choice of sampling scheme is a modelling decision** in that it effects what we recover from fitting models such as the one described above. In other words:

> **For the purposes of Box's loop, we should take into account our choice of sampling scheme as part of our modeling step, moreso than as part of our inference step.**

## Choices of sampling scheme we'll consider ##

Although there is a range of sampling schemes we could potentially consider, as we only partially observe network edges (with the edges taking on continuous variables), we consider two straighforward examples to implement:

1. **p-sampling** [2] - We sample edges uniformly at random from the entirety of those observed with probability $p$.
2. **Selecting edges by random walks** [3] - A uniform random walk of length $K$ is performed on the undirected version of the observed network - from a node $i$, we move to the next via selecting an edge connected to $i$ uniformly at random - and then we remove any self-crossings from the graph. (Note: for reasons of implementation, this can only be used to explore nodes which have a positive in and out degree.)

This specifies a collection of edges; the corresponding vertices will simply consisting only of those from which there is a sampled edge incident from it.

## Fitting Model 2 ##

### Sampling scheme choice ###

We try the following sampling schemes with the following parameters:

* p-sampling with $p = 0.005$;
* random-walk sampling by starting $15$ random walks at random locations of length $20$.

These both correspond to sampling (roughly) 300 edges per iteration.

### Hyperparameter choice ###

We use the following hyperparameters when fitting the model:

* We set the embedding dimension $d=4$ - the behavior of users should be explainable by a low rank structure;
* We set $\sigma_0^2 = 1/\sqrt{d}$, so the magnitude of the embedding vectors is approximately $1$;
* We vary $\tau_0^2 \in \{0.316, 0.1, 0.0316\}$, as this controls the strength of the matrix factorization approximation and the logistic regression term;
* We optimize over the variational parameters using ADAM and a learning rate of $0.0005$ for all parameters, stopping after $9000$ iterations for p-sampling (which is roughly $30$ epochs) and $3000$ for the random walk procedure (this is partly for computation time reasons, and also as the random walk method will far more heavily sample the more information edges).

In [35]:
# This is simple a demo which shows that the code does indeed work
import matplotlib.pyplot as plt

if demo:
    data = bitcoin.BitcoinOTC()
    # Note: krein = False corresponds to Model 2
    # krein = True corresponds to Model 2.5
    embed_args_dict = {'embed_dim': 4, 'krein': False,
                       'omega_model_scale': 1/np.sqrt(4), 
                       'obs_scale': 0.1,
                       'learning_rate': 0.0005, 
                       'num_iters': 500, 'logging': True}
    # Comment out depending on what you want to try
    #sample_args_dict = {'sample_str': 'p-sampling',
    #                    'sample_args': {'sample_prob': 0.005}}
    sample_args_dict = {'sample_str': 'random-walk', 
                        'sample_args': {'start_num': 15, 'walk_length': 20}}
    
    # Run the algorithm
    vp_dict, elbo = embed.embed(data, embed_args_dict, sample_args_dict)
    
    store_params = {}
    store_params['0.1'] = vp_dict
    
    plt.plot(elbo)
    plt.ylabel('ELBO')
    plt.show()


## Results and criticism for Model 2 ##

We'll examine the following measures/checks for each hyperparameter choice and choice of the embedding method:

* We will examine the ELBO, simply to ensure that convergence seems to have actually occured;
* The calibration curve produced by using the estimated probabilities $p_i = \sigma( \mu_0 + \langle \beta_0, \mu_i \rangle )$ for each user;
* A plot of the empirical distribution of the inner products $\langle \omega_i, \omega_j \rangle$ versus that of $\mathrm{logit}(a_{ij})$, to see if the method seems to recover user behaviour; and
* A PCA projection of the learned embedding vectors into 2D, so we can help visualize what was actually learned by the method.

In [ ]:
obs_scale_list = [0.0316]
store_params = {}
store_elbo = {}

data = bitcoin.BitcoinOTC()

# Comment out depending on the choice of sampling scheme
#sample_args_dict = {'sample_str': 'p-sampling', 'sample_args': {'sample_prob': 0.005}}
sample_args_dict = {'sample_str': 'random-walk', 'sample_args': {'start_num': 15, 'walk_length': 20}}

if ~demo:
    for obs_scale in obs_scale_list:
        # Define dictionary
        embed_args_dict = {'embed_dim': 4, 'krein': True, 'omega_model_scale': 1/np.sqrt(4), 'obs_scale': obs_scale,
                           'learning_rate': 0.0005, 'num_iters': 3000, 'logging': True}
        
        # Get learned variational parameters, store them for now
        vp_dict, elbo = embed.embed(data, embed_args_dict, sample_args_dict)
        store_params[str(obs_scale)] = vp_dict
        store_elbo[str(obs_scale)] = elbo

In [ ]:
import matplotlib.pyplot as plt

fig = plt.figure(figsize=(8, 6))
ax = fig.add_subplot(1, 1, 1)

for key, elbo in store_elbo.items():
    ax.plot(elbo, label='obs_scale: {}'.format(key))
    
ax.set_xlabel('per 500 iterations')
ax.set_ylabel('ELBO')
ax.set_yscale('log')
ax.set_title('ELBO, sampling: {}'.format(sample_args_dict['sample_str']))
ax.legend(loc="upper right")

if ~demo:
    fig.savefig('img/elbo_sampling_{}_krein_{}.png'.format(
                sample_args_dict['sample_str'], 
        str(embed_args_dict['krein'])), bbox_inches='tight')

![title](img/elbo_sampling_p-sampling_krein_False.png)

We'll begin by examining what happens for p-sampling; looking at the ELBO, it appears that the ELBO has begun to mostly flatten out, suggesting that we have likely approximately converged. Examining the calibration curves...

In [ ]:
import matplotlib.pyplot as plt
from sklearn.calibration import calibration_curve
from sklearn.metrics import roc_curve

def sigmoid(x):
    return np.exp(x)/(1+np.exp(x))

fig = plt.figure(figsize=(7, 7))
ax = fig.add_subplot(1, 1, 1)
ax.plot([0, 1], [0, 1], "k:", label="Perfectly calibrated")

for key, vp_dict in store_params.items():
    logits = vp_dict['mu_loc'] + torch.mv(vp_dict['omega_loc'].t(), vp_dict['beta_loc'])
    probs = sigmoid(logits.detach().numpy())
    
    frac_positive, mean_pred = calibration_curve(data.gt[data.nodes_test], probs[data.nodes_test], n_bins=10)
    ax.plot(mean_pred, frac_positive, 's-',
             label='Est. prob (obs_scale: {})'.format(key))

ax.set_xlabel('Actual values')
ax.set_ylabel('Fraction of positives')
ax.set_ylim([-0.04, 1.04])
ax.legend(loc="upper left")
ax.set_title('Calibration curve, sampling: {}'.format(sample_args_dict['sample_str']))

if ~demo:
    fig.savefig('img/calib_curve_sampling_{}_krein_{}.png'.format(
                sample_args_dict['sample_str'], str(embed_args_dict['krein'])), bbox_inches='tight')

![title](img/calib_curve_sampling_p-sampling_krein_False.png)

....we see something very worrying - if anything, it looks like the estimate probabilities are calibrated to the wrong class! We'll have a quick look at the empirical distribution of the inner products:

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

for key, vp_dict in store_params.items():
    # Calculate the gram matrix
    if embed_args_dict['krein']:
        embed_dim = embed_args_dict['embed_dim']
        gram_pos = torch.mm(vp_dict['omega_loc'][:int(embed_dim/2), :].t(),
                            vp_dict['omega_loc'][:int(embed_dim/2), :])
        gram_neg = torch.mm(vp_dict['omega_loc'][int(embed_dim/2):, :].t(),
                            vp_dict['omega_loc'][int(embed_dim/2):, :])
        gram = gram_pos - gram_neg
    else:
        gram = torch.mm(vp_dict['omega_loc'].t(), vp_dict['omega_loc'])
    
    gram_dist = gram[data.edge_index[0, :], data.edge_index[1, :]].detach().numpy()
    
    # Plot distribution of gram_dist versus that of data.edge_weight_logit
    fig = plt.figure(figsize=(6, 6))
    ax = fig.add_subplot(1, 1, 1)
    ax.set_xlabel('Ratings on logit scale', fontsize=10)
    ax.set_ylabel('Frequency', fontsize=10)
    ax.set_title('Sampling scheme: {}, obs_scale: {}'.format(sample_args_dict['sample_str'], key),
                 fontsize=12)
    
    sns.distplot(data.edge_weight_logit.numpy(), hist=True, bins=20,
                 label='Observed distribution',
                 color='#949494', ax=ax)
    
    sns.distplot(gram_dist, hist=True, kde=True, bins=20,
                 kde_kws={'linewidth': 3},
                 label='Predicted distribution',
                 color='#0173B2', ax=ax)
    
    ax.legend(loc='upper right')
    
    if ~demo:
        fig.savefig('img/rating_dist_sampling_{}_obs_scale_{}_krein_{}.png'.format(
                    sample_args_dict['sample_str'], key, str(embed_args_dict['krein'])), bbox_inches='tight')

![title](img/rating_dist_sampling_p-sampling_krein_False.png)

It appears that for all three methods, we doesn't seem to be too far off the population distribution of ratings. If we look at the PCA of the embeddings, we finally see what the issue is:

In [ ]:
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA

good_users = []
bad_users = []
for ind in data.nodes_with_gt:
    if data.gt.numpy()[ind] == 1:
        good_users.append(ind)
    else:
        bad_users.append(ind)

for key, vp_dict in store_params.items():
    # Do PCA on the mean embedding vectors
    pca = PCA(n_components=2)
    embed_vectors = vp_dict['omega_loc'].t().detach().numpy()
    embed_pca = pca.fit_transform(embed_vectors)
    
    fig = plt.figure(figsize=(6, 6))
    ax = fig.add_subplot(1, 1, 1)
    ax.set_xlabel('PCA 1', fontsize=10)
    ax.set_ylabel('PCA 2', fontsize=10)
    ax.set_title('Sampling scheme: {}, obs_scale: {}'.format(sample_args_dict['sample_str'], key),
                 fontsize=12)
    
    ax.scatter(embed_pca[:, 0], embed_pca[:, 1], c='#949494', alpha = 1, s=6)
    ax.scatter(embed_pca[good_users, 0], embed_pca[good_users, 1],
               c='#0173B2', alpha=0.8, s=12, label='Trustworthy')
    ax.scatter(embed_pca[bad_users, 0], embed_pca[bad_users, 1],
               c='#D55E00', alpha=0.8, s=12, label='Untrustworthy')
    
    ax.grid()
    ax.legend()
    
    if demo:
        plt.show()
    else:
        fig.savefig('img/embed_pca_sampling_{}_obs_scale_{}_krein_{}.png'.format(
                    sample_args_dict['sample_str'], key, str(embed_args_dict['krein'])), bbox_inches='tight')

![title](img/embed_pca_sampling_p-sampling_krein_False.png)

As these plots show, we've effectively learnt nothing from the above embedding procedure. If we look at similar plots for the random walk procedure (which we investigate for only one hyperparameter), we see little different (with the exception of some evidence that the random walk classifier actually manages to distingush between the trustworthy and non-trustworthy users in the test set).

![title](img/sampling_random-walk_krein_False.png)

# Model 2.5: Removing the non-negative definiteness assumption on $\mathrm{logit}(a_{ij})$ #

In the previous model, we tried to learn embedding vectors $\omega_i$ such that $\mathrm{logit}(a_{ij}) \approx \langle \omega_i, \omega_j \rangle$ - this forces this matrix to be non-negative definite. To see how this is an implicit constraint on the behavior of users, consider a more simplisitic model where user $i$ gives rating $j$ either a ratings of $+1$ and $-1$, doing the latter with probability $p_{e_i, e_j}$. With the above assumption, this effectively boils down to a requirement that the matrix

$$ \begin{pmatrix} p_{00} & p_{01} \\ p_{10} & p_{11} \end{pmatrix} $$

is non-negative definite. This is equivalent to requiring that $p_{00} p_{11} \geq p_{01} p_{10}$ - this places some constraint on the behavior of trustworthy and non-trustworthy users. We therefore investigate what happens if we allow for some degree of flexibility, by instead setting 

$$ p\big( \{a_{ij} \,:\, i \neq j \} \,|\, \omega_1, \ldots, \omega_N \big)  = \prod_{i \neq j} \mathrm{Normal}\big( \mathrm{logit}(a_{ij}) \,\big|\, \langle \omega_i, B \omega_j \rangle, \tau_0^2 \big) $$

in the likelihood, where $B = \mathrm{diag}\big( I_{d/2}, - I_{d/2} \big)$ - i.e, we allow for the logit ratings matrix to have negative eigenvalues.

## Fitting Model 2.5 and model criticism ##

We show similar plots to before; unfortunately, for p-sampling we do not see much of a difference to before. (There was also no real change using the random walk method, so we do not bother plotting the relevant images.)

![title](img/elbo_sampling_p-sampling_krein_True.png)

![title](img/calib_curve_sampling_p-sampling_krein_True.png)

![title](img/rating_dist_sampling_p-sampling_krein_True.png)

![title](img/embed_pca_sampling_p-sampling_krein_True.png)

# Conclusion #

Unfortunately, we don't seem to have made much progress. The main reasons as to why I suspect why are more to do with some particular aspects of the data: namely that the network is sparse (few users have a large in/out degree, and the average number of edges per users is relatively small). In hingsight, it'd be more interesting to evaluate this on a slightly more dense network, as given that we learn a meaningful embedding, the logistic regression part should work reasonably well. Although I've only iterated through a few steps of Box's loop, I want to give a few more examples of how one could keep on expanding on the above models:

* Using a more flexible classifier on the embedding vectors (e.g a small neural network).
* Incorporating other covariate information (e.g some of the summary statistics of the rating time distributions) when forming the embeddings/as part of the logistic regression step.
* Handling possible asymmetry in ratings by assigning each user an embedding $\omega_i = (\omega_i^{(out)}, \omega_i^{(in)})$ and then trying to create an approximation $\mathrm{logit}(a_{ij}) \approx \langle \omega_i^{(out)}, \omega_j^{(in)} \rangle$.
* Investigate choosing a different 'link function' to convert the ratings to a continuous spectrum.
* Try this out on different datasets!

Moreover, it would be desirable to have an embedding model which updates as new ratings come in, although this would be substanatially more complicated.

# References

[1] - Veitch, Victor & Austern, Morgane & Zhou, Wenda & Blei, David & Orbanz, Peter. (2019). "Empirical Risk Minimization and Stochastic Gradient Descent for Relational Data". 22nd International Conference on Artificial Intelligence and Statistics.

[2] - V. Veitch and D. M. Roy. 2016. “Sampling and Estimation for (Sparse) Exchangeable Graphs”. arXiv: 1611.00843.

[3] - A. Grover, J. Leskovec. 2016. "node2vec: Scalable Feature Learning for Networks". ACM SIGKDD International Conference on Knowledge Discovery and Data Mining (KDD).